In [14]:
import numpy as np
import pandas as pd
import os
import shutil 
import xarray as xr
import glob 
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

In [15]:
#%% set up data paths and output directories

#path for python parser script
parserpath = r'D:\Active_Projects\LOWE_IMOS_WaveBuoys\Data\SofarSpotter\SofarParser\parser_v1.12.0'
parser = 'parser_v1.12.0.py'
#meta data about spotter
buoyID = 'SPOT-1018'

#output path
dpath = r'D:\Active_Projects\HANSEN_UWA-UNSW_Linkage\Data\CSIRO_Collab\DEP02_FEB-MMM'
outpath = os.path.join(dpath,buoyID)
datapath = os.path.join(dpath,buoyID,'raw')




In [16]:
#%% determine number of unique file numbers for chunking data into management number of files
files = os.listdir(datapath)
dum = []
[dum.append(i[0:4]) for i in files]
file_nums = np.unique(np.array(dum))
chunk_size= 10

#Create tmp directory for moving files to process
tmp_path = os.path.join(datapath, 'tmp')
os.mkdir(tmp_path)
shutil.copyfile(os.path.join(parserpath, parser), os.path.join(tmp_path,parser))

#could use this on read_csv - fix later 
def sofar_date_parser(x1,x2,x3,x4,x5,x6,x7):
    t = datetime(int(x1),int(x2),int(x3),int(x4),int(x5),int(x6))+timedelta(seconds=float(x7)/1000)
    return t

In [17]:
#%% loop over chunks and build dataframes 
import warnings
warnings.filterwarnings('ignore')

os.chdir(tmp_path)
for i in range(0, len(file_nums), chunk_size):
    print('Processing files ' + str(i) + ' to ' + str(i+chunk_size) + ' out of ' + str(len(file_nums)))
    chunk = file_nums[i:i+chunk_size]
    #move files in chunk to temp folder
    for j in chunk:      
        for k in files:
            if k[0:4]==j:
                shutil.copyfile(os.path.join(datapath, k), os.path.join(tmp_path, k))
    #now process    
    runfile(os.path.join(tmp_path,parser))
    
    #check whether parser generated subdirectories or not
    if os.path.exists('bulkparameters.csv')==False:
        ff = os.listdir(tmp_path)
        for checkfile in ff:
            if os.path.isdir(checkfile)==True:
                
                dumblk = pd.read_csv(os.path.join(tmp_path, checkfile,'bulkparameters.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
    
                dumdisp = pd.read_csv(os.path.join(tmp_path, checkfile,'displacement.csv'),
                                      parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                
                dumloc = pd.read_csv(os.path.join(tmp_path, checkfile, 'location.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser) 
                duma1 = pd.read_csv(os.path.join(tmp_path,checkfile,'a1.csv'),
                                    parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                duma2 = pd.read_csv(os.path.join(tmp_path,checkfile,'a2.csv'),
                                    parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumb1 = pd.read_csv(os.path.join(tmp_path,checkfile,'b1.csv'),
                                    parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumb2 = pd.read_csv(os.path.join(tmp_path,checkfile,'b2.csv'),
                                    parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumSxx = pd.read_csv(os.path.join(tmp_path,checkfile,'Sxx.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumSyy = pd.read_csv(os.path.join(tmp_path,checkfile,'Syy.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumSzz = pd.read_csv(os.path.join(tmp_path,checkfile,'Szz.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumsst = pd.read_csv(os.path.join(tmp_path,checkfile,'sst.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)                                                      
    else:
        dumblk = pd.read_csv('bulkparameters.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)          
        dumdisp = pd.read_csv('displacement.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)            
        dumloc = pd.read_csv('location.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)    
        duma1 = pd.read_csv('a1.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
        duma2 = pd.read_csv('a2.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
        dumb1 = pd.read_csv('b1.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
        dumb2 = pd.read_csv('b2.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
        dumSxx = pd.read_csv('Sxx.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
        dumSyy = pd.read_csv('Syy.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
        dumSzz = pd.read_csv('Szz.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
        dumsst = pd.read_csv('sst.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)              
    
    if i == 0:
        bulkparams = dumblk
        displacements = dumdisp
        locations = dumloc
        a1 = duma1
        a2 = duma2
        b1 = dumb1
        b2 = dumb2
        Sxx = dumSxx
        Syy = dumSyy
        Szz = dumSzz
        sst = dumsst
    else:
        bulkparams = pd.concat([bulkparams,dumblk],ignore_index=True)
        displacements = pd.concat([displacements,dumdisp],ignore_index=True)
        locations = pd.concat([locations,dumloc],ignore_index=True)
        a1 = pd.concat([a1,duma1],ignore_index=True)
        a2 = pd.concat([a2,duma2],ignore_index=True)
        b1 = pd.concat([b1,dumb1],ignore_index=True)
        b2 = pd.concat([b2,dumb2],ignore_index=True)
        Sxx = pd.concat([Sxx,dumSxx],ignore_index=True)
        Syy = pd.concat([Syy,dumSyy],ignore_index=True)
        Szz = pd.concat([Szz,dumSzz],ignore_index=True)                                
      
    #delete files from tmp directory
    for i in os.listdir(tmp_path):
        if i[-3:]=='csv':
            os.remove(i)
        elif i[-3:]=='CSV':
            os.remove(i)
        elif os.path.isdir(i):
            shutil.rmtree(i)    
    
#delete tmp directory if last file      
print('Finished processing ' + buoyID)

Processing files 0 to 10 out of 2420
Concatenating all FLT files:
- 0007_FLT.CSV (File 1 out of 5)
- 0008_FLT.CSV (File 2 out of 5)
- 0014_FLT.CSV (File 3 out of 5)
- 0015_FLT.CSV (File 4 out of 5)
- 0016_FLT.CSV (File 5 out of 5)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0008_SPC.CSV (File 1 out of 4)
- 0014_SPC.CSV (File 2 out of 4)
- 0015_SPC.CSV (File 3 out of 4)
- 0016_SPC.CSV (File 4 out of 4)
Processing spotter spectral output
Concatenating all SYS files:
- 0007_SYS.CSV (File 1 out of 10)
- 0008_SYS.CSV (File 2 out of 10)
- 0009_SYS.CSV (File 3 out of 10)
- 0010_SYS.CSV (File 4 out of 10)
- 0011_SYS.CSV (File 5 out of 10)
- 0012_SYS.CSV (File 6 out of 10)
- 0013_SYS.CSV (File 7 out of 10)
- 0014_SYS.CSV (File 8 out of 10)
- 0015_SYS.CSV (File 9 out of 10)
- 0016_SYS.CSV (File 10 out of 10)
Concatenating all LOC files:
- 0007_LOC.CSV (File 1 out of 5)
- 0008_LOC.CSV (File 2 out of 5)
- 0014_LOC.CSV (File 3 o

- 0046_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Processing files 40 to 50 out of 2420
Concatenating all FLT files:
- 0047_FLT.CSV (File 1 out of 10)
- 0048_FLT.CSV (File 2 out of 10)
- 0049_FLT.CSV (File 3 out of 10)
- 0050_FLT.CSV (File 4 out of 10)
- 0051_FLT.CSV (File 5 out of 10)
- 0052_FLT.CSV (File 6 out of 10)
- 0053_FLT.CSV (File 7 out of 10)
- 0054_FLT.CSV (File 8 out of 10)
- 0055_FLT.CSV (File 9 out of 10)
- 0056_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0047_SPC.CSV (File 1 out of 10)
- 0048_SPC.CSV (File 2 out of 10)
- 0049_SPC.CSV (File 3 out of 10)
- 0050_SPC.CSV (File 4 out of 10)
- 0051_SPC.CSV (File 5 out of 10)
- 0052_SPC.CSV (File 6 out of 10)
- 0053_SPC.CSV (File 7 out of 10)
- 0054_SPC.CSV (File 8 out of 10)
- 0055_SPC.CSV (File 9 out of 10)
- 0056_SPC.CSV (File 10 out of 10)
Processing spotter spectral output
Concatenating all SYS files:


- 0079_SST.CSV (File 3 out of 10)
- 0080_SST.CSV (File 4 out of 10)
- 0081_SST.CSV (File 5 out of 10)
- 0082_SST.CSV (File 6 out of 10)
- 0083_SST.CSV (File 7 out of 10)
- 0084_SST.CSV (File 8 out of 10)
- 0085_SST.CSV (File 9 out of 10)
- 0086_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Processing files 80 to 90 out of 2420
Concatenating all FLT files:
- 0087_FLT.CSV (File 1 out of 10)
- 0088_FLT.CSV (File 2 out of 10)
- 0089_FLT.CSV (File 3 out of 10)
- 0090_FLT.CSV (File 4 out of 10)
- 0091_FLT.CSV (File 5 out of 10)
- 0092_FLT.CSV (File 6 out of 10)
- 0093_FLT.CSV (File 7 out of 10)
- 0094_FLT.CSV (File 8 out of 10)
- 0095_FLT.CSV (File 9 out of 10)
- 0096_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0087_SPC.CSV (File 1 out of 10)
- 0088_SPC.CSV (File 2 out of 10)
- 0089_SPC.CSV (File 3 out of 10)
- 0090_SPC.CSV (File 4 out of 10)
- 0091_SPC.CSV (File 5 out of 1

Concatenating all GPS files:
  No _GPS data files available.
Concatenating all SST files:
- 0117_SST.CSV (File 1 out of 10)
- 0118_SST.CSV (File 2 out of 10)
- 0119_SST.CSV (File 3 out of 10)
- 0120_SST.CSV (File 4 out of 10)
- 0121_SST.CSV (File 5 out of 10)
- 0122_SST.CSV (File 6 out of 10)
- 0123_SST.CSV (File 7 out of 10)
- 0124_SST.CSV (File 8 out of 10)
- 0125_SST.CSV (File 9 out of 10)
- 0126_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Processing files 120 to 130 out of 2420
Concatenating all FLT files:
- 0127_FLT.CSV (File 1 out of 10)
- 0128_FLT.CSV (File 2 out of 10)
- 0129_FLT.CSV (File 3 out of 10)
- 0130_FLT.CSV (File 4 out of 10)
- 0131_FLT.CSV (File 5 out of 10)
- 0132_FLT.CSV (File 6 out of 10)
- 0133_FLT.CSV (File 7 out of 10)
- 0134_FLT.CSV (File 8 out of 10)
- 0135_FLT.CSV (File 9 out of 10)
- 0136_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0127_

- 0160_SST.CSV (File 4 out of 10)
- 0161_SST.CSV (File 5 out of 10)
- 0162_SST.CSV (File 6 out of 10)
- 0163_SST.CSV (File 7 out of 10)
- 0164_SST.CSV (File 8 out of 10)
- 0165_SST.CSV (File 9 out of 10)
- 0166_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Processing files 160 to 170 out of 2420
Concatenating all FLT files:
- 0167_FLT.CSV (File 1 out of 10)
- 0168_FLT.CSV (File 2 out of 10)
- 0169_FLT.CSV (File 3 out of 10)
- 0170_FLT.CSV (File 4 out of 10)
- 0171_FLT.CSV (File 5 out of 10)
- 0172_FLT.CSV (File 6 out of 10)
- 0173_FLT.CSV (File 7 out of 10)
- 0174_FLT.CSV (File 8 out of 10)
- 0175_FLT.CSV (File 9 out of 10)
- 0176_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0167_SPC.CSV (File 1 out of 10)
- 0168_SPC.CSV (File 2 out of 10)
- 0169_SPC.CSV (File 3 out of 10)
- 0170_SPC.CSV (File 4 out of 10)
- 0171_SPC.CSV (File 5 out of 10)
- 0172_SPC.CSV (File 6 out of

- 0198_SST.CSV (File 2 out of 10)
- 0199_SST.CSV (File 3 out of 10)
- 0200_SST.CSV (File 4 out of 10)
- 0201_SST.CSV (File 5 out of 10)
- 0202_SST.CSV (File 6 out of 10)
- 0203_SST.CSV (File 7 out of 10)
- 0204_SST.CSV (File 8 out of 10)
- 0205_SST.CSV (File 9 out of 10)
- 0206_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Processing files 200 to 210 out of 2420
Concatenating all FLT files:
- 0207_FLT.CSV (File 1 out of 10)
- 0208_FLT.CSV (File 2 out of 10)
- 0209_FLT.CSV (File 3 out of 10)
- 0210_FLT.CSV (File 4 out of 10)
- 0211_FLT.CSV (File 5 out of 10)
- 0212_FLT.CSV (File 6 out of 10)
- 0213_FLT.CSV (File 7 out of 10)
- 0214_FLT.CSV (File 8 out of 10)
- 0215_FLT.CSV (File 9 out of 10)
- 0216_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0207_SPC.CSV (File 1 out of 10)
- 0208_SPC.CSV (File 2 out of 10)
- 0209_SPC.CSV (File 3 out of 10)
- 0210_SPC.CSV (File 4 out of

- 0239_SST.CSV (File 3 out of 10)
- 0240_SST.CSV (File 4 out of 10)
- 0241_SST.CSV (File 5 out of 10)
- 0242_SST.CSV (File 6 out of 10)
- 0243_SST.CSV (File 7 out of 10)
- 0244_SST.CSV (File 8 out of 10)
- 0245_SST.CSV (File 9 out of 10)
- 0246_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Processing files 240 to 250 out of 2420
Concatenating all FLT files:
- 0247_FLT.CSV (File 1 out of 10)
- 0248_FLT.CSV (File 2 out of 10)
- 0249_FLT.CSV (File 3 out of 10)
- 0250_FLT.CSV (File 4 out of 10)
- 0251_FLT.CSV (File 5 out of 10)
- 0252_FLT.CSV (File 6 out of 10)
- 0253_FLT.CSV (File 7 out of 10)
- 0254_FLT.CSV (File 8 out of 10)
- 0255_FLT.CSV (File 9 out of 10)
- 0256_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0247_SPC.CSV (File 1 out of 10)
- 0248_SPC.CSV (File 2 out of 10)
- 0249_SPC.CSV (File 3 out of 10)
- 0250_SPC.CSV (File 4 out of 10)
- 0251_SPC.CSV (File 5 out of

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
#%% clean up for export
os.chdir(outpath)
shutil.rmtree(tmp_path)
############# BULKPARAMS ###############################################
bulkparams = bulkparams.rename(columns={bulkparams.columns[0]:'time'})
bulkparams = bulkparams.set_index('time').tz_localize(tz='UTC')
bulkparams.to_csv(os.path.join(outpath,buoyID+'_bulkparams.csv'))

########### DISPLACEMENTS ################################################
displacements = displacements.rename(columns={displacements.columns[0]:'time'})
displacements = displacements.set_index('time').tz_localize(tz='UTC')
displacements.to_csv(os.path.join(outpath,buoyID+'_displacements.csv'))

########### LOCATIONS ######################################################
locations = locations.rename(columns={locations.columns[0]:'time'})
locations = locations.set_index('time').tz_localize(tz='UTC')
locations.to_csv(os.path.join(outpath,buoyID+'_locations.csv'))

############ A1 ###########################################################
a1 = a1.rename(columns={a1.columns[0]:'time'})
a1 = a1.set_index('time').tz_localize(tz='UTC')
a1.to_csv(os.path.join(outpath,buoyID+'_a1.csv'))

############ A2 ###########################################################
a2 = a2.rename(columns={a2.columns[0]:'time'})
a2 = a2.set_index('time').tz_localize(tz='UTC')
a2.to_csv(os.path.join(outpath,buoyID+'_a2.csv'))

############ b1 ###########################################################
b1 = b1.rename(columns={b1.columns[0]:'time'})
b1 = b1.set_index('time').tz_localize(tz='UTC')
b1.to_csv(os.path.join(outpath,buoyID+'_b1.csv'))

############ b2 ###########################################################
b2 = b2.rename(columns={b2.columns[0]:'time'})
b2 = b2.set_index('time').tz_localize(tz='UTC')
b2.to_csv(os.path.join(outpath,buoyID+'_b2.csv'))

############ Sxx ###########################################################
Sxx = Sxx.rename(columns={Sxx.columns[0]:'time'})
Sxx = Sxx.set_index('time').tz_localize(tz='UTC')
Sxx.to_csv(os.path.join(outpath,buoyID+'_Sxx.csv'))

############ Syy ###########################################################
Syy = Syy.rename(columns={Syy.columns[0]:'time'})
Syy = Syy.set_index('time').tz_localize(tz='UTC')
Syy.to_csv(os.path.join(outpath,buoyID+'_Syy.csv'))

############ Szz ###########################################################
Szz = Szz.rename(columns={Szz.columns[0]:'time'})
Szz = Szz.set_index('time').tz_localize(tz='UTC')
Szz.to_csv(os.path.join(outpath,buoyID+'_Szz.csv'))

############ SST ###########################################################
sst = sst.rename(columns={sst.columns[0]:'time'})
sst = sst.set_index('time').tz_localize(tz='UTC')
sst.to_csv(os.path.join(outpath,buoyID+'_sst.csv'))